# Устанавливаем пакеты и включаем их

In [ ]:
# !pip3 install tensorflow numpy matplotlib

In [ ]:
import cv2
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras import layers

# Загружаем датасет

In [ ]:
# !unzip training_data.zip
# !unzip validation_data.zip

In [ ]:
train_dir = pathlib.Path("training_data/")
test_dir = pathlib.Path("validation_data/")
val_dir = pathlib.Path("validation_data/")
img_height = 256
img_width = 256

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir)

class_names = dict(zip(train_ds.class_names, range(len(train_ds.class_names))))
num_classes = len(class_names)

Found 296 files belonging to 46 classes.
Found 44 files belonging to 46 classes.
Found 44 files belonging to 46 classes.


# Преобразование датасета

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function = mobilenet_v2.preprocess_input,
    rotation_range = 32,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

In [ ]:
test = train_generator.flow_from_directory(
    test_dir,
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = False
)
train = train_generator.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 123
)

validation = train_generator.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 123
)

Found 44 images belonging to 46 classes.
Found 296 images belonging to 46 classes.
Found 44 images belonging to 46 classes.


# МЯУ МОДЕЛЬ

In [ ]:
mobilenet_ = MobileNetV2(
    input_shape = (img_height, img_width, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'avg'
)
mobilenet_.trainable = False

In [ ]:
inputs = mobilenet_.input
x = Dense(128, activation = 'relu')(mobilenet_.output)
x = Dense(128, activation = 'relu')(x)
outputs = Dense(num_classes, activation = 'softmax')(x)

mobilenet = Model(inputs = inputs, outputs = outputs)

In [ ]:
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    mode = 'min',
    patience = 2,
    verbose = 1,
    restore_best_weights = True
)

checkpoint = ModelCheckpoint('/meow/',
                             monitor = 'val_loss',
                             mode = 'min',
                             save_best_only = True)

callbacks = [early_stopping, checkpoint]


In [ ]:
mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

history = mobilenet.fit(
    train, validation_data = validation,
    batch_size = 32,
    epochs = 15,
    callbacks = callbacks
)

Epoch 1/15
10/10 [==============================] - ETA: 0s - loss: 3.6565 - accuracy: 0.1149

10/10 [==============================] - 52s 5s/step - loss: 3.6565 - accuracy: 0.1149 - val_loss: 3.5748 - val_accuracy: 0.1364
Epoch 2/15
10/10 [==============================] - ETA: 0s - loss: 2.7742 - accuracy: 0.3851

10/10 [==============================] - 57s 6s/step - loss: 2.7742 - accuracy: 0.3851 - val_loss: 3.1442 - val_accuracy: 0.2955
Epoch 3/15
10/10 [==============================] - ETA: 0s - loss: 1.9702 - accuracy: 0.5912

10/10 [==============================] - 49s 5s/step - loss: 1.9702 - accuracy: 0.5912 - val_loss: 2.6587 - val_accuracy: 0.4318
Epoch 4/15
10/10 [==============================] - ETA: 0s - loss: 1.2973 - accuracy: 0.7297

10/10 [==============================] - 47s 5s/step - loss: 1.2973 - accuracy: 0.7297 - val_loss: 2.1053 - val_accuracy: 0.6136
Epoch 5/15
10/10 [==============================] - ETA: 0s - loss: 0.8848 - accuracy: 0.8142

10/10 [==============================] - 50s 5s/step - loss: 0.8848 - accuracy: 0.8142 - val_loss: 1.9209 - val_accuracy: 0.6364
Epoch 6/15
10/10 [==============================] - ETA: 0s - loss: 0.5956 - accuracy: 0.8919

10/10 [==============================] - 48s 5s/step - loss: 0.5956 - accuracy: 0.8919 - val_loss: 1.5311 - val_accuracy: 0.7045
Epoch 7/15
10/10 [==============================] - 26s 3s/step - loss: 0.3742 - accuracy: 0.9223 - val_loss: 1.6930 - val_accuracy: 0.7273
Epoch 8/15
10/10 [==============================] - ETA: 0s - loss: 0.2786 - accuracy: 0.9527

10/10 [==============================] - 47s 5s/step - loss: 0.2786 - accuracy: 0.9527 - val_loss: 1.4894 - val_accuracy: 0.6818
Epoch 9/15
10/10 [==============================] - ETA: 0s - loss: 0.1951 - accuracy: 0.9797

10/10 [==============================] - 45s 5s/step - loss: 0.1951 - accuracy: 0.9797 - val_loss: 1.4381 - val_accuracy: 0.7500
Epoch 10/15
10/10 [==============================] - 25s 2s/step - loss: 0.1664 - accuracy: 0.9764 - val_loss: 1.5039 - val_accuracy: 0.7273
Epoch 11/15
10/10 [==============================] - 24s 2s/step - loss: 0.1372 - accuracy: 0.9865 - val_loss: 1.4590 - val_accuracy: 0.7500
Epoch 11: early stopping


In [ ]:
(eval_loss, eval_accuracy) = mobilenet.evaluate(test)

2/2 [==============================] - 2s 369ms/step - loss: 1.5106 - accuracy: 0.7273


# Прогоним на тестовой фигне

In [ ]:
# получаем предсказанные значения от тестовых изображений
pred = mobilenet.predict(test)
# получаем номер класса с максимальным весом
pred = np.argmax(pred,axis=1)
# сопоставляем классы
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]
# получаем предсказанные классы
y_test = [labels[k] for k in test.classes]
# оцениваем точность
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(f'Accuracy on the test set: {100*acc:.2f}%')

2/2 [==============================] - 3s 403ms/step
Accuracy on the test set: 65.91%


In [ ]:
mobilenet.save('meow/tmodel.h5')